上下文窗口内的词共现可以携带丰富的语义信息.例如,在⼀个大型语料库中,"固体"比"气体"更有可能与“冰”共现,但"气体"⼀词与"蒸汽"的共现频率可能比与"冰"的共现频率更高.此外,可以预先计算此类共现的全局语料库统计数据:这可以提高训练效率.

### 共现矩阵

设共现矩阵为$X$,其元素为$X_{i,j}$.

$X_{i,j}$表示在整个语料库中,单词$i$和单词$j$同时出现在一个窗口中的次数

设有语料库:

i love you but you love him i am sad

如果采用一个窗口宽度为5(左右长度都为2)的统计窗口,那么就有以下窗口内容:

| 窗口标号| 	中心词| 	窗口内容| 
| ---- | ---- | ---- |
| 0| 	i| 	i love you| 
| 1| 	love| 	i love you but| 
| 2| 	you| 	i love you but you| 
| 3| 	but| 	love you but you love| 
| 4| 	you| 	you but you love him| 
| 5| 	love| 	but you love him i| 
| 6| 	him| 	you love him i am| 
| 7| 	i| 	love him i am sad| 
| 8| 	am| 	him i am sad| 
| 9| 	sad| 	i am sad| 

窗口0、1长度小于5是因为中心词左侧内容少于2个,同理窗口8、9长度也小于5.

以窗口5为例说明如何构造共现矩阵:

中心词为love,语境词为but、you、him、i;则执行:

$X_{\text {love }, \text { but }}+=1$

$X_{\text {love, you }}+=1$

$X_{\text {love }, \text { him }}+=1$

$X_{\text {love }, i}+=1$

使用窗口将整个语料库遍历一遍,即可得到共现矩阵$X$:

| count | i    | love | you  | but  | you  | love | him  | i    | am   | sad  |
| ----- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| i     | 0    | 4    | 2    | 0    | 2    | 4    | 2    | 0    | 2    | 2    |
| love  | 4    | 0    | 4    | 4    | 5    | 0    | 2    | 4    | 0    | 0    |
| you   | 2    | 4    | 2    | 4    | 2    | 4    | 2    | 4    | 0    | 0    |
| but   | 0    | 4    | 4    | 0    | 4    | 4    | 0    | 4    | 0    | 0    |
| you   | 2    | 5    | 2    | 4    | 2    | 4    | 0    | 0    | 0    | 0    |
| love  | 4    | 0    | 4    | 4    | 4    | 0    | 0    | 0    | 0    | 0    |
| him   | 2    | 2    | 2    | 0    | 0    | 0    | 0    | 0    | 2    | 0    |
| i     | 0    | 4    | 4    | 4    | 0    | 0    | 0    | 0    | 0    | 2    |
| am    | 2    | 0    | 0    | 0    | 0    | 0    | 2    | 0    | 0    | 2    |
| sad   | 2    | 0    | 0    | 0    | 0    | 0    | 0    | 2    | 2    | 0    |

### 模型推导

定义几个符号:

$$
X_{i}=\sum_{j=1}^{N} X_{i, j}
$$

表示共现矩阵中单词$i$那一行的和;

$$
P_{i, k}=\frac{X_{i, k}}{X_{i}}
$$

表示单词$k$出现在单词$i$语境中的概率(共现概率)

$$
\operatorname{ratio}_{i, j, k}=\frac{P_{i, k}}{P_{j, k}}
$$


注意到$\operatorname{ratio}_{i, j, k}$存在以下规律:

| $\operatorname{ratio}_{i, j, k}$ | 单词$j,k$相关 | 单词$j,k$不相关 |
| ------------------------------ | ---- | ---- |
| 单词$ i,k$相关 | $ \approx $1 | >>1 |
| 单词$i,k$相关 | <<1 | $ \approx $1 |

由此可见,共现概率的比值能够直观地表达词与词之间的关系

设用词向量$\mathrm{v}_i, \mathrm{v}_j, \mathbf{v}_k$拟合$\operatorname{ratio}_{i, j, k}$的函数为$f(\mathrm{v}_i,\mathrm{v}_j,\mathrm{v}_k)$,即

$$
\frac{P_{i, k}}{P_{j, k}} \approx f\left(\mathrm{v}_{i}, \mathrm{v}_{j}, \mathrm{v}_{k}\right)
$$

因为共现概率的比值$\operatorname{ratio}_{i, j, k}$为标量,故要求$f$为标量函数

函数$f$构造思路:

1. 要考虑单词$i$和单词$j$的关系,故可以在$f\left(\mathrm{v}_{i}, \mathrm{v}_{j}, \mathrm{v}_{k}\right)$中构造:$\mathrm{v}_{i} - \mathrm{v}_{j}$
2. 函数$f$为标量函数,故可以构造:$  (\mathrm{v}_{i} - \mathrm{v}_{j})^T \mathrm{v}_k $
3. 若交换词索引$i$和$j$的位置,则有$\frac{P_{i, k}}{P_{j, k}} \frac{P_{j, k}}{P_{i, k}} = 1 $,则函数$f$满足$f(x)f(-x)=1$,故可以设定$f(x) = \exp(x)$

此时有:

$$
\begin{aligned}
\frac{P_{i, k}}{P_{j, k}} &\approx \exp \left(\left(\mathrm{v}_{i}-\mathrm{v}_{j}\right)^{T} \mathrm{v}_{k}\right) \\
\frac{P_{i, k}}{P_{j, k}} &\approx \frac{\exp \left(\mathrm{v}_{i}^{T} \mathrm{v}_{k}\right)}{\exp \left(\mathrm{v}_{j}^{T} \mathrm{v}_{k}\right)}
\end{aligned}
$$

### 损失函数推导

设定

$$
\begin{aligned}
\exp \left(\mathrm{v}_{i}^{T} \mathrm{v}_{k}\right) &\approx \alpha P_{i,k } \\
\exp \left(\mathrm{v}_{j}^{T} \mathrm{v}_{k}\right) &\approx \beta P_{j,k }
\end{aligned}
$$

其中$\alpha,\beta$为常数,分子分母形式相同,这里只考虑$\exp \left(\mathrm{v}_{i}^{T} \mathrm{v}_{k}\right) \approx \alpha P_{i,k } $,等式两边取对数,得:

$$
\mathrm{v}_{i}^{T} \mathrm{v}_k \approx \log \alpha + \log X_{i,k} - \log X_{i}
$$

$ \log \alpha - X_{i}$与$k$无关,因此可以使用附加的偏置项来拟合$ \log \alpha - X_{i}$,如中心词偏置$b_k$和上下⽂词偏置$c_i$,即:

$$
\mathrm{v}_{i}^{T} \mathrm{v}_k  + b_k + c_i  \approx \log X_{i,k}
$$

于是损失函数为:

$$
J=\sum_{i, j}^{N}\left(\mathrm{v}_{i}^{T} \mathrm{v}_{j}+b_{k}+c_{j}-\log \left(X_{i, j}\right)\right)^{2}
$$

通过对加权平方误差,使用权重函数$h(X_{x,j})$替换每个损失项的权重,即有

$$
J=\sum_{i, j}^{N} h(X_{i,j}) \left(\mathrm{v}_{i}^{T} \mathrm{v}_{j}+b_{k}+c_{j}-\log \left(X_{i, j}\right)\right)^{2}
$$

其中$h(x)$在$[0; 1]$的间隔内递增(考虑到共现频率较低的词对,其相对重要性应该较低)

glove词向量的训练见:https://github.com/stanfordnlp/GloVe